In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

True

In [5]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.chains import LLMMathChain, LLMChain

# Create tools for the agent
################### first custom tool #############

#create the WMI serach tool

#Load the documents
from langchain_community.document_loaders import DirectoryLoader
#loader = TextLoader("./bella_vista.txt")
DATA_PATH = "mydata"
loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
docs = loader.load()

#Split the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

print("####### FIRST 10 docs ############")
for i in range(10):
    print("\n")
    print(documents[i])

print("####### LAST 10 docs ############")
x = documents[::-1]
i = 0
while (i < 10):
    print("\n")
    print(x[i])
    i+= 1

print("############## # of Documents ##################")
print(len(documents))

####### FIRST 10 docs ############


page_content='PUSH POSSIBLETM

WAREHOUSE MANAGEMENT AND TRANSPORT EXECUTION FOR IBM i

RELEASE NOTES 2024

0

WMi AND TEi RELEASE NOTES

Copyright and Disclaimer

Copyright © 2024 Manhattan Associates, Inc. All rights reserved.

This documentation, as well as the software described in it, is furnished under license and may be used or copied only in accordance with the terms of such license. The information in this documentation is furnished for informational use only, is subject to change without notice, and should not be construed as a commitment by Manhattan Associates, Inc. (“Manhattan”). No third-party patent liability is assumed with respect to the use of the information contained herein. While every precaution has been taken in the preparation of this documentation, Manhattan assumes no responsibility for errors or omissions.' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='EXCEPT WHERE EXPRESSLY PROVIDED OTHERWISE, ALL CONTENT, MATER

In [6]:
# I'm importing the necessary libraries and initializing the Pinecone client
import os
import pinecone

from langchain_community.vectorstores import Pinecone

pc = pinecone.Pinecone()

In [7]:
# deleting all indexes
indexes = pc.list_indexes().names()
for i in indexes:
    print('Deleting all indexes ... ', end='')
    pc.delete_index(i)
    print('Done')

Deleting all indexes ... Done


In [8]:
# creating an index
from pinecone import ServerlessSpec
index_name = 'wmsearch-gemini'
if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
        name=index_name,
        dimension=768, #for Gemini
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print('Index created! 😊')
else:
    print(f'Index {index_name} already exists!')

Creating index wmsearch-gemini
Index created! 😊


In [9]:
#Initiatlise the Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain_google_vertexai import VertexAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#embeddings = GoogleGenerativeAIEmbeddings(model="multimodalembedding")


In [10]:
#Generate the embeddings
# inserting the embeddings into the index and returning a new Pinecone vector store object. 
vectorstore = Pinecone.from_documents(documents, embeddings, index_name=index_name)

In [11]:
#See the index
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 314}},
 'total_vector_count': 314}

In [12]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class WMSearchTool(BaseTool):
    name = "WM search"
    description = "useful when you need to answer questions about WM product"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        #docs = store.get_relevant_documents(query)
        docs = store.invoke(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

################## custom tool end ####################


In [13]:
# 1. Python REPL Tool (for executing Python code)
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain, LLMChain


python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

In [18]:
# Import the necessary class for interacting with Google's generative AI models
from langchain_google_genai import ChatGoogleGenerativeAI

# Create an instance of the LLM, using the 'gemini-pro' model and set creativity level
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0)  


# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

#Another math tool as REPL not working as expected
from langchain.chains import LLMMathChain, LLMChain
problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions."
)

In [29]:
# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
#llm=OpenAI()

#Answer the following questions in GERMAN as best you can.
# Define a template for answering questions
template = '''
Answer the following questions as best you can.
Questions: {q}
'''
# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')

# Combine the tools into a list
tools = [WMSearchTool(), math_tool, python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [31]:
#question = 'Does WMi support json interfaces?'
#question = 'Who is the current prime minister of the UK?'
#question = 'what is the answer to 3 ** 3.9'
question = 'Who is Sumit Gupta of Manhattan Associates, pls get from Linkedin?'
#question = 'Tell me about Napoleon Bonaparte early life'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active Supply Chain'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active® Warehouse Management'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active Supply Chain, enabled by default?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active® Warehouse Management, enabled by default?'


output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
I should search LinkedIn for Sumit Gupta of Manhattan Associates.
Action: LinkedIn Search
Action Input: Sumit Gupta of Manhattan AssociatesLinkedIn Search is not a valid tool, try one of [WM search, Calculator, Python REPL, Wikipedia, DuckDuckGo Search].I should search LinkedIn for Sumit Gupta of Manhattan Associates.
Action: DuckDuckGo Search
Action Input: Sumit Gupta of Manhattan Associates LinkedInCheck out professional insights posted by Sumit Gupta, Technology &amp; General Management | Supply Chain | Payments | Big Data Analytics MC Interview: We can build in India for the world, says CoinDCX's Sumit Gupta The company is bullish on its prospects in India and expects India to take the lead in the crypto and web 3.0 sector ... In 2014, Sumit returned to Sony where he served as a Software Engineer for a year. In January 2016, he co-founded ListUp™, a place for people to buy & sell pre-owned electronics. [2] CoinDCX. In March 2018, Sumit Gupta 